In [ ]:
import numpy as np
import pandas as pd
import re
import os
from google.colab import drive
drive.mount('/content/drive')
import pickle
import numpy as np
import pandas as pd
import sklearn
from copy import deepcopy
from stellargraph import StellarGraph
import time

In [ ]:
all_sub_adjMat = np.load("/content/drive/My Drive/correlation_matrices.npy")
sub_names = np.load("/content/drive/My Drive/filenamelist.npy", allow_pickle= True)
print(len(all_sub_adjMat), len(sub_names))

In [ ]:
subnames = [str(i)[5:15] for i in sub_names]
print(len(subnames), subnames[:5])

In [ ]:
with open('/content/xlabel.csv') as f:
    content = f.readlines()
content = [x.strip() for x in content]
content.pop(-1)
print(len(content))

In [ ]:
df = pd.read_csv('/content/xlabel.csv') 
label = df['Group'].tolist()
subject = df['Subject'].tolist()
print(len(label),label[:5], subject[:5])

In [ ]:
adni90_feature = []
adni90_labels = []
adni90_adjmat = []

for x,i in enumerate(subnames):
    ind = subject.index(i)
    adni90_labels.append(label[ind])
    adni90_feature.append(all_sub_adjMat[x])

for i in adni90_feature:
    temp = np.zeros((90,90))
    for x in range(90):
        for y in range(90):
            if i[x][y] > 0.15:
                temp[x][y] = 1
    adni90_adjmat.append(list(temp))
            
    
print(len(adni90_labels), len(adni90_adjmat), len(adni90_feature))

In [ ]:
all_sg = [[] for i in range(152)]

for count, sub in enumerate(adni90_adjmat):
    np.fill_diagonal(np.array(sub), np.nan)
    df = pd.DataFrame(sub)
    df = df.stack().reset_index()
    df.columns = ['a','b','c']
    isedge = df['c'] == 1
    dfn = df[isedge]
    pairs = list(zip(dfn['a'], dfn['b']))
    for i in pairs:
            a, b = i
            if (b,a) not in all_sg[count]:
                all_sg[count].append(i)

In [ ]:
print(len(all_sg))
print(len(all_sg[0]))

In [ ]:
all_sg_lr = [[] for _ in range(152)]

for i,x in enumerate(all_sg):
    all_sg_lr[i] = [[],[]]
    for pair in x:
        a, b = pair
        all_sg_lr[i][0].append(a)
        all_sg_lr[i][1].append(b)

In [ ]:
# for featue

all_sub_fea = []
for i in adni90_feature:
    fead = pd.DataFrame(i)
    all_sub_fea.append(fead)

In [ ]:
graphs = []

for i,one_sglr in enumerate(all_sg_lr):
    dic = {"source": one_sglr[0],"target":one_sglr[1]}
    one = pd.DataFrame(dic)
    graphs.append(StellarGraph(all_sub_fea[i],one))

In [ ]:
print(graphs[1].info())

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:

################### EMCI Vs LMCI  #############################
# graph_labels = []
# rem_ad = []
# exp1 = []
# for c,i in enumerate(adni90_labels):
#     if i == 'EMCI':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#     elif i == 'LMCI':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)
   

################## AD Vs NC ################################
# graph_labels = []
# rem_ad = []
# exp1 = []
# coo = 0
# for c,i in enumerate(adni90_labels):
#     if i == 'AD':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#         coo += 1
#     elif i == 'CN':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)


################  AD Vs MCI ###############################
# graph_labels = []
# rem_ad = []
# exp1 = []
# coo = 0
# for c,i in enumerate():
#     if i == 'EMCI' or i == 'LMCI' or i == 'MCI':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#         coo += 1
#     elif i == 'AD':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)
   
   


########### MCI Vs NC ##########################
graph_labels = []
rem_ad = []
exp1 = []
coo = 0
for c,i in enumerate(adni90_labels):
    if i == 'EMCI' or i == 'LMCI' or i == 'MCI':
        graph_labels.append(1)
        exp1.append(graphs[c])
        coo += 1
    elif i == 'CN':
        graph_labels.append(-1)
        exp1.append(graphs[c])
    else:
        rem_ad.append(c)
    

In [ ]:
print(len(exp1), len (graph_labels))
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
#####################################################################################3

In [ ]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets
import sklearn
from sklearn import model_selection
from sklearn.metrics import balanced_accuracy_score
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.metrics
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
generator = PaddedGraphGenerator(graphs=exp1)

In [ ]:
def create_graph_classification_model(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[128, 128, 128, 128],
        activations=["relu", "relu","relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=256, activation="relu")(x_out)
    predictions = Dense(units=128, activation="relu")(predictions)
    
    predictions = Dense(units=64, activation="relu")(predictions)
    
    predictions = Dense(units=32, activation="relu")(predictions)
    
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
#     model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"]) 
    model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc",tf.keras.metrics.AUC()])

    return model

In [ ]:
epochs = 200  # maximum number of training epochs
folds = 30 # the number of folds for k-fold cross validation
n_repeats = 5  # the number of repeats for repeated k-fold cross validation

In [ ]:
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)

In [ ]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=0, callbacks=[es],
    )
    # calculate performance on the test data and return along with history
    
    test_metrics = model.evaluate(test_gen, verbose=0)
#     print(model.metrics_names)
    test_acc = test_metrics[model.metrics_names.index("acc")]
#     test_bacc = test_metrics[3]
    test_auc = test_metrics[2]
    

    return history, test_acc, test_auc

In [ ]:
def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )

    return train_gen, test_gen

In [ ]:
test_accs = []
# test_raccs = []
# test_baccs = []
test_aucs = []
start = time.time()


stratified_folds = model_selection.RepeatedStratifiedKFold(
    n_splits=folds, n_repeats=n_repeats
).split(graph_labels, graph_labels)

for i, (train_index, test_index) in enumerate(stratified_folds):
    print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
    train_gen, test_gen = get_generators(
        train_index, test_index, graph_labels, batch_size=30
    )

    model = create_graph_classification_model(generator)
    history, acc, auc = train_fold(model, train_gen, test_gen, es, epochs)
    
#     history, acc = train_fold(model, train_gen, test_gen, es, epochs)

    test_accs.append(acc)
#     test_raccs.append(racc)
    test_aucs.append(auc)

end = time.time()
print(end - start)

In [ ]:
print(
    f"Accuracy over all folds mean: {np.mean(test_accs)*100:.3}% and std: {np.std(test_accs)*100:.2}% and auc: {np.mean(test_aucs)}"
)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(test_accs)
plt.xlabel("Accuracy")
plt.ylabel("Count")

In [ ]:
# Accuracy over all folds mean: 75.5% and std: 2.1e+01% and auc: 0.7890625
# Accuracy over all folds mean: 90.0% and std: 2.2e+01% and auc: 0.9080645161290323
# Accuracy over all folds mean: 85.1% and std: 1.4e+01% and auc: 0.8333333367301572
